#Decision Tree

each item has two continuous features
class label is {0, 1}



In [8]:
import numpy as np
import pandas as pd

df = pd.read_csv('./data/D1.txt', sep='\s+', names=['x1', 'x2', 'y'])
df.columns[:-1]

Index(['x1', 'x2'], dtype='object')

In [ ]:
def determine_candidate_numeric_splits(D: pd.DataFrame, X: str):
    return

def determine_candidate_splits(D: pd.DataFrame):
    C = []
    for feature in D.columns[:-1]:
        C.append(determine_candidate_numeric_splits(D, feature))
    return C

def make_subtree(D: pd.DataFrame):
    candidate_splits = determine_candidate_splits(D)
    # if stopping criteria:
    #   make a leaf node N
    #   determine class label of N
    # else:
    #   make internal node N
    #   S = FindBestSplit(D, candidate_splits)
    #   for each group k of S
    #       Dk := subset of training data in group k
    #       kth child of N := make_subtree(Dk)
    # return subtree rooted at N
    return

dtree = make_subtree(df[:])